# Summary:

**Tutorial Difficulty: Beginner**

This notebook documents:
* How to define a simple pipeline defined by operations that are contained in docker containers
* An example of a pipeline where each step communicates its outputs to the next step by writing them to a file locally and having kubeflow pipelines transfer results between steps as JSON strings
* A map-reduce workflow pattern, where we:
    * break our work into many small pieces that can be done in parallel (map), and then
    * aggregate the product of that work back to some final result (reduce)

To bring it all together, we apply these techniques to compute an estimate of pi. 

# Pipeline to estimate pi, in the most ridiculously parallel way possible

This pipeline estimates pi by repeating the process of:

* Picking a random location inside a 2x2 square centered on the origin
* Checking whether or not that point also resides inside a unit circle centered on the origin
* Assigning a value to this point:
    * value = 1 if the point is inside the circle (red)
    * value = 0 if the point is outside the circle (blue)

By doing this repeatedly and taking 4x the average value over all repetitions, we obtain an estimate of pi

![Parallel Monte Carlo](images/Pi.png)

We implement this procedure using the map-reduce pattern by:
* **Map:** Generating N **sample** operations which pick the point and assign it a value of 0/1.  Note that each **sample** operation is given a different random seed to ensure it picks a different point in the square
* **Reduce:** Combining all **sample** results in an **average** step which then returns the estimate of pi

The pipeline, as visualized in kubeflow pipelines, looks like this:

![The pipeline](images/kf-pipeline.png)

Where the top row of **sample** operations all feed to the single **average** step on the second row.  

# Set up our Project

In [15]:
import json
import re
from utilities import validate_kfp_name, validate_bucket_name

#################################
### Configure your variables ####
#################################

# Number of parallel sample steps we run
SAMPLES = 15

# Name of our experiment in kubeflow
# Experiment name can contain alphanumeric characters, hyphens, or underscores
EXPERIMENT_NAME = "compute-pi"
assert validate_kfp_name(EXPERIMENT_NAME)

# Names and container images for our pipeline operations
# Pipeline operations have the same name restrictions as experiments
SAMPLE_IMAGE_PATH = f"k8scc01covidacr.azurecr.io/blair-kf-pipeline-pi-sample:v10"
SAMPLE_PIPELINE_OP_NAME = "one-pi-estimate"
assert validate_kfp_name(SAMPLE_PIPELINE_OP_NAME)

AVERAGE_IMAGE_PATH = f"k8scc01covidacr.azurecr.io/blair-kf-pipeline-pi-average:v10"
AVERAGE_PIPELINE_OP_NAME = "aggregate-pi-estimate"
assert validate_kfp_name(AVERAGE_PIPELINE_OP_NAME)

########################################
### This gets fed into the map step ####
########################################
# Define a generator that creates the numeric random seed for each sample step
def seeds(how_many=SAMPLES):
    """ Define the seeds for the algorithms """
    for i in range(how_many):
        yield { "seed" : 3 * i }

# Define the pipeline

This is where we define all operations in our pipeline, as well as how they chain together.  Pipelines are defined by separate, typically single purpose, operations (or steps).  Each pipeline operation could be used once, multiple times, etc., and might depend on results from upstream steps.

## Define the pipeline operations

Our pipeline here has two steps, both of which are defined in docker containers (paths to those containers were specified above and are used below).  Each step is a factory function that returns ContainerOp's.  These ContainerOps are then used to define the actual pipeline next.

For this example, the containers are already built and pushed to ```k8scc01covidacr.azurecr.io``` (see ```SAMPLE_IMAGE_PATH``` and ```AVERAGE_IMAGE_PATH``` above).  Each container has a small shell script to do the work for that operation (check out the scripts at ```./sample/sample.sh``` and ```./average/average.sh``` to see how they work).  This notebook defines two kubeflow pipeline operation (```sample_op``` and ```average_op```) that specify how kubeflow interacts with those containers (how to call them, what args to provide, what to do with their outputs, ...).  

For this example, we pass arguments like our random seed as JSON strings rather than bare numbers.  It isn't really necessary here, but gives an example of how one can pass a non-trivial data type.  This can be extended to complicated dictionaries with many keys and even serialized (JSONified) versions of objects like dataframes.  

Side note: Technically ```sample_op``` and ```average_op``` are factories that return ContainerOp instances, and kubeflow pipelines uses those ContainerOp instances to build a pipeline, but if none of that makes sense its ok...

In [16]:
from kfp import dsl
import itertools

def sample_op(params):
    """
    Factory for "sample" pipeline operation
    
    Operations created by this factory invoke the SAMPLE step by invoking
    a docker container that includes ./sample/sample.sh.  sample.sh accepts
    a random seed.
    
    The result of this operation will be an out.json file with contents:
        { "x" : point_x_coord, "y" : point_y_coord, "result" : 0_or_1 }
    This result will be passed back as _sample_op_result.output
    
    Args:
        params (str): JSON string of a dict that has a seed value as key "seed", eg:
                        '{"seed": 5, "some_ignored_key": "value_doesnt_matter"}

    Returns:
        JSON string of out.json
    """
    # Assemble arguments passed to the script called by the container op
    arguments = ["--params", params]
    
    # And to the ContainerOp constructor
    containerop_kwargs = dict(
        name=SAMPLE_PIPELINE_OP_NAME,
        image=f'{SAMPLE_IMAGE_PATH}',
        arguments=arguments,
        # Specify where kubeflow will get output from
        file_outputs={'data': "./output/out.json"},
    )
    
    # Return the actual Container Op, with additional memory and cpu constraints
    return dsl.ContainerOp(
        **containerop_kwargs,
    ).set_memory_request(
        "100M"
    ).set_memory_limit(
        "150M"
    ).set_cpu_request(
        "0.1"
    ).set_cpu_limit(
        "1"
    )

In [17]:
def average_op(jsons=None):
    """
    Factory for "average" pipeline operation
        
    Operations created by this factory invoke the AVERAGE step by invoking
    a docker container that includes ./average/average.sh.  average.sh accepts
    results from one or more SAMPLE steps as JSON strings and returns output
    as a JSON string

    Generates an output file out.json with contents:
        { "pi": estimate_of_pi, "samples": number_of_samples }
    This result is passed back as _sample_op_result.output
    
    Args:
        jsons (list): List of JSON strings of output from one or more 
                      "sample" steps.  Each must have a "result" key with the value
                      from the sample step.

    Returns:
        JSON result file is captured by Kubeflow and saved as an artifact
    """
    if not jsons:
        raise ValueError("Must specify one or more json string inputs")
    
    # Assemble arguments passed to the script called by the container op
    arguments = []
    if jsons:
        json_args = list(itertools.chain.from_iterable([("--json", j) for j in jsons]))
        arguments += json_args
    
    # And to the ContainerOp constructor
    containerop_kwargs = dict(
        name=AVERAGE_PIPELINE_OP_NAME,
        image=f'{AVERAGE_IMAGE_PATH}',
        arguments=arguments,
        # Specify where to get output from
        file_outputs={'data': "./output/out.json"},
    )
    
    # Return the actual Container Op
    return dsl.ContainerOp(**containerop_kwargs)

## Define the pipeline

We define the pipeline here as a python function wrapped in the @dsl.pipeline decorator.  This function, in our case compute_pi(), defines the logic for how all the steps within the pipeline chain together.  In our case, it tells kubeflow pipelines to run N **sample** operations in parallel, and run a single **average** operation that consumes output from all the **sample** operations.  

This dependency of **average** on **sample**s is what lets kfp know the order in which to run things.  

In [18]:
######################################
### You can change below this      ###
### Create the pipeline            ###
######################################
@dsl.pipeline(
    name="Estimate Pi",
    description='Estimate Pi using a Map-Reduce pattern'
)
def compute_pi():
    """Compute Pi"""

    # Create arguments for each "sample" operation in the pipeline
    # Each operation gets its own seed and a path in minio to store its output
    # params is passed as a json string with just {'seed': value_of_seed}
    sample_args = [{'params': json.dumps(param)} for (i, param) in enumerate(seeds())]

    # Create a sample operation for each arg
    sample_ops = [sample_op(**kwarg) for kwarg in sample_args]

    # Define the average operation which consumes output from all the sample_ops
    _average_op = average_op(
        jsons=[s.output for s in sample_ops],
    )

It is important to understand here that while ```compute_pi``` describes the pipeline in python code, most of the computation is not done when we run the above block.  Calling ```sample_op``` does not do a **sample** operation, it creates a ContainerOp that tells kubeflow pipelines to run a **sample** operation when running the pipeline.  And when we do something like:
```
_average_op = average_op(
    jsons=[s.output for s in sample_ops],
)
```

```s.output``` is not the actual output of a **sample** operation, it is a placeholder that tells kubeflow pipelines "when you get to this part in the pipeline, insert the output that you've previous computed for this **sample** operation here".  This way you can pipe data from one pipeline step to the next without having to actually compute it now.

Finally, we translate our compute_pi function into a zipped yaml definition of the pipeline.  This zip file is how we tell kubeflow pipelines exactly what to run for your pipeline.  Download and take a look inside to get a better understanding!

In [19]:
###############################################
### DON'T EDIT:                             ###
### Create the pipeline description for kfp ###
###############################################
from kfp import compiler
experiment_yaml_zip = EXPERIMENT_NAME + '.zip'
compiler.Compiler().compile(
    compute_pi,
    experiment_yaml_zip
)
print(f"Exported pipeline definition to {experiment_yaml_zip}")

Exported pipeline definition to compute-pi.zip


# Ready to roll! Let's run this pipeline!

In [20]:
###################################
### DON'T EDIT:                 ###
### Create the Experiment       ###
###################################
import kfp
client = kfp.Client()
exp = client.create_experiment(name=EXPERIMENT_NAME)

In [21]:
###############################################
### DON'T EDIT:                             ###
### Run the pipeline                        ###
###############################################
import time
run = client.run_pipeline(
    exp.id,
    EXPERIMENT_NAME + '-' + time.strftime("%Y%m%d-%H%M%S"),
    EXPERIMENT_NAME + '.zip',
)

# Collect results

To see the pipeline running, click the link above.  To access the JSON string returned by the Average step, click on that step in the pipeline and look in the output artifact as shown below.

![pipeline with results](images/kf-pipeline_with_result.png)

Note that this method of returning a result is likely not that useful for most problems.  See other other demos for saving results to minio or other locations.